<a href="https://colab.research.google.com/github/drwitt/BME_590_Tensorflow_Deep_Learning/blob/master/Example_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install tensorflow_datasets
!pip3 install tensorflow-gpu==2.0.0-rc0
!nvidia-smi

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
ds_corrupted = tfds.load('mnist_corrupted/shot_noise', shuffle_files=False, split=tfds.Split.TRAIN)
ds_identity = tfds.load('mnist_corrupted/identity', shuffle_files=False, split=tfds.Split.TRAIN)
ds_corrupted = ds_corrupted.map(lambda x: x['image'])
ds_identity = ds_identity.map(lambda x: x['image'])
ds = tf.data.Dataset.zip((ds_corrupted, ds_identity))

In [0]:
BATCH_SIZE = 128

for x,y in ds: break
fig, axes = plt.subplots(1,2)
axes[0].imshow(np.squeeze(x.numpy()), cmap='gray')
axes[1].imshow(np.squeeze(y.numpy()), cmap='gray')
ds = ds.batch(BATCH_SIZE).repeat()

In [0]:
# Convolutional
inputs = tf.keras.layers.Input(shape=(28,28,1))

x = tf.keras.layers.Conv2D(48, (7,7), activation='relu', padding='same')(inputs)


# block 1
xa = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
xb = tf.keras.layers.Conv2D(16, (5,5), activation='relu', padding='same')(x)
xc = tf.keras.layers.Conv2D(16, (7,7), activation='relu', padding='same')(x)
x_combined = tf.keras.layers.Concatenate()([xa, xb, xc])
x = tf.keras.layers.Add()([x, x_combined]) # residual
x = tf.keras.layers.MaxPooling2D()(x)

# block 2
xc = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
xc = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(xc)
x = tf.keras.layers.Add()([x, xc]) # residual
x = tf.keras.layers.MaxPooling2D()(x)

# block 3
xc = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
xc = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(xc)
x = tf.keras.layers.Add()([x, xc]) # residual
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(1, (3,3), activation='relu', padding='same')(x)
model = tf.keras.Model(inputs, x)
model.compile('adam', loss='mse')
model.fit(ds, steps_per_epoch=50000//128, epochs=20)

In [0]:
import tensorflow as tf

model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)
model.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
